<a href="https://colab.research.google.com/github/reachrkr/llm-genai/blob/main/LCEL_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
    langchain \
    openai\
    docarray \
   tiktoken \
   faiss-cpu

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<your-open-ap-key>"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Give me small report about {topic}"
)
model = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    #openai_api_key=openai_api_key
)

llm = ChatOpenAI(temperature=0)
output_parser = StrOutputParser()

## Runnable

The syntax follows standard Linux piping, introduced in Python. The | operator takes output from the left and feeds it into the function on the right.


When the Python interpreter encounters the | operator connecting two objects, such as obj1 | obj2, it invokes the __or__ method of object obj2 by passing object obj1
 as an argument. This implies that the following patterns are interchangeable:



Keeping this in mind, we can create a Runnable class that takes a function, transforming it into a chainable function using the pipe operator |. This forms the essence of LCEL.

In [ ]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

def add_one(x):
    return x + 1

def add_two(x):
    return x + 2



# run them using the object approach
chain_or = Runnable(add_one).__or__(Runnable(add_two))
print(chain_or(3))
# run  using a|b
chain_pipe = Runnable(add_one)| (Runnable(add_two))
print(chain_pipe(3))

6
6


In [ ]:
###

##  LCEL prompt
Combine a prompt and a language model to create a chain. Take user input, add it to a prompt, pass it to a model, and get the raw model output. Example:

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)

# and run
out = chain.run(topic="Large Multimodal Model")
print(out)

Large Multimodal Model (LMM) is a state-of-the-art artificial intelligence model developed by OpenAI. It is designed to understand and generate human-like text based on a given prompt, while also being capable of processing and generating images. LMM combines the power of language models with computer vision capabilities, making it a versatile tool for various applications.

One of the key features of LMM is its ability to understand and generate text in a multimodal context. It can process both textual and visual information, allowing it to generate detailed and coherent responses that incorporate both words and images. This multimodal approach enables LMM to provide more comprehensive and contextually relevant outputs.

LMM is trained on a massive dataset that includes a wide range of text and image data from the internet. This extensive training allows the model to learn patterns, understand context, and generate high-quality outputs. The training process involves optimizing the mod

In [ ]:
# Using LCEL

lcel_chain = prompt | model | output_parser

# and run
out = lcel_chain.invoke({"topic": "Large Multimodal Model"})
print(out)

Large Multimodal Model (LMM) is a state-of-the-art artificial intelligence model developed by OpenAI. It is designed to understand and generate human-like text based on a given prompt, while also being capable of processing and generating images. LMM combines the power of language models with computer vision capabilities, making it a versatile tool for various applications.

One of the key features of LMM is its ability to understand and generate text in a multimodal context. It can process both textual and visual information, allowing it to generate detailed and coherent responses that incorporate both words and images. This multimodal approach enables LMM to provide more comprehensive and contextually relevant outputs.

LMM is trained on a massive dataset that includes a wide range of text and image data from the internet. This extensive training allows the model to learn patterns, understand context, and generate high-quality outputs. The training process involves optimizing the mod

To regulate text generation in your chain, you have the option to incorporate stop sequences. In this configuration, the process of generating text concludes when a newline character is detected

In [ ]:
chain = prompt | model.bind(stop=["\n"])
result = chain.invoke({"topic": "Large Multimodal Model"})
print(result)

content='Large Multimodal Model (LMM) is a state-of-the-art artificial intelligence model developed by OpenAI. It is designed to understand and generate human-like text based on a given prompt, while also being capable of processing and generating images. LMM combines the power of language models with computer vision capabilities, making it a versatile tool for various applications.'


LCEL facilitates the attachment of function call information to your chain, enhancing the functionality and providing valuable context during text generatio.This example attaches function call information to generate a summary.

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

functions = [
    {
        "name": "summary",
        "description": "A summary",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "LMM summary"},
                "punchline": {
                    "type": "string",
                    "description": "Summary",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "summary"}, functions=functions)   | JsonOutputFunctionsParser()
result = chain.invoke({"topic": "Large Multimodal Model"}, config={})
print(result)

{'setup': 'Large Multimodal Model (LMM) is a state-of-the-art model that combines multiple modalities, such as text, image, and audio, to perform various tasks.', 'punchline': 'LMM has achieved impressive results in tasks like image captioning, visual question answering, and speech recognition, surpassing previous models in terms of performance and versatility.'}


LCEL enables the creation of retrieval-augmented generation chains, merging retrieval and language generation steps for a comprehensive and sophisticated approach to content creation

In [ ]:

from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

# Create a vector store and retriever
vectorstore = FAISS.from_texts(
    [" ability of a digital computer or computer-controlled robot to perform tasks commonly associated with intelligent beings","Artificially induced intelligence"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

# Define templates for prompts
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

# Create a retrieval-augmented generation chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

result = chain.invoke("what is Artificial intelligence?")
print(result)

Artificial intelligence refers to the ability of a digital computer or computer-controlled robot to perform tasks commonly associated with intelligent beings.


# Multiple Chains


The integration of Runnables allows for the concatenation of multiple chains, enabling a seamless connection between distinct processes for enhanced and cohesive text generation.

In this instance, a branching and merging chain is applied to construct a rationale, analyze its merits and drawbacks, and then generate a conclusive response.


In [ ]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("is this {city} caputal of this country?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

result = chain2.invoke({"city": "Rome", "language": "spanish"})
print(result)

Lo siento, pero necesito más contexto para responder tu pregunta con precisión. ¿Podrías especificar a qué país te refieres?


**Branching and Merging**

LCEL facilitates the splitting and merging of chains through RunnableMaps. Here's an example illustrating branching and merging:

In [ ]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

result = chain.invoke({"input": "Agile"})
print(result)

While Agile methodology has numerous benefits, it is important to consider the potential drawbacks and assess whether they align with the specific project requirements and organizational culture. The lack of predictability and potential for increased complexity can be challenging for organizations with strict deadlines or limited resources. Additionally, the high level of dependency on customer involvement can slow down the development process if the customer is not readily available or lacks a clear understanding of their requirements. Scaling Agile methodologies across large organizations or complex projects can also be a significant undertaking. The prioritization of working software over comprehensive documentation can make it difficult to track changes or troubleshoot issues. Additionally, the flexible nature of Agile can sometimes lead to scope creep if additional features or requirements are added without proper evaluation. Effective team collaboration and communication are cruc

## Batch, Stream & Async

**Batch processing** is enhanced through LangChain's Expression Language, streamlining LLM queries by executing multiple tasks concurrently. LangChain's batch functionality optimizes inputs by employing parallel LLM calls, ensuring efficient and improved performance in interactions with the LLM model.

In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Given the items: {items}, What games can I play")
chain = prompt | model |  StrOutputParser()
response = chain.batch([{"items": "bat, ball, gloves"},{"items":"stick, ball, gloves, pads"}])
print(response)

['With the items bat, ball, and gloves, you can play various games, including:\n\n1. Baseball: Use the bat to hit the ball and play a game of baseball, either individually or with a team.\n2. Softball: Similar to baseball, but with a larger ball and a different set of rules.\n3. Cricket: Use the bat and ball to play the sport of cricket, either in a formal setting or casually with friends.\n4. Catch: Use the gloves to play a simple game of catch, throwing the ball back and forth with a partner.\n5. T-Ball: Use the bat and ball to play a simplified version of baseball, where the ball is placed on a stationary tee for easier hitting.\n6. Wiffle Ball: Play a modified version of baseball using a lightweight plastic ball and a plastic bat, suitable for smaller spaces.\n7. Kickball: Although primarily played with a larger ball, you can use the bat to kick the ball instead, adding a twist to the traditional game.\n8. Handball: Use the gloves to play a game of handball, hitting the ball agains

**Stream functionality** in LangChain facilitates immediate data flow, making it well-suited for dynamic chatbots and live-stream applications. ChefBot exemplifies this capability by seamlessly streaming information, eliminating any wait time and showcasing the power of LangChain in dynamic contexts.

In [ ]:
chain = prompt | model
for s in chain.stream({"items": "ball ,jersey, shoes"}):print(s.content, end="")

There are several games you can play with the items ball, jersey, and shoes. Here are a few options:

1. Basketball: Use the ball and shoes to play a game of basketball. You can wear the jersey to represent your favorite team.

2. Soccer or Football: Use the ball and shoes to play a game of soccer or football. The jersey can be worn to represent your team or just for fun.

3. Dodgeball: Use the ball and shoes to play a game of dodgeball. The jersey can help differentiate teams or just add to the fun.

4. Baseball or Softball: Use the ball and shoes to play a game of baseball or softball. The jersey can be worn to represent your team or just for a sporty look.

5. Kickball: Use the ball and shoes to play a game of kickball. The jersey can be worn for team identification or just for a sporty vibe.

6. Volleyball: Use the ball and shoes to play a game of volleyball. While you may not need the jersey for this game, you can wear it for a sporty look or to represent a team.

These are just a

By leveraging ainvoke and await methods, a seamless **asynchronous execution** is achieved. This empowers tasks to operate independently, substantially enhancing responsiveness and application speed within the realm of asynchronous capabilities.






In [ ]:
response = await chain.ainvoke({"items": "shuttle cock, bat"})
print(response)

content='With these items, you can play the game of badminton.'


In [ ]:
# Parallelize steps

RunnableParallel, also known as RunnableMap, simplifies the simultaneous execution of multiple Runnables. It seamlessly returns the output of these Runnables as a map, providing an efficient and straightforward approach to parallel processing.







In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI()
story_chain = ChatPromptTemplate.from_template("tell me a story about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(story=story_chain, poem=poem_chain)

map_chain.invoke({"topic": "goofy"})

{'story': AIMessage(content="Once upon a time in the whimsical land of Disney, there lived a lovable and clumsy character named Goofy. Goofy was a tall, anthropomorphic dog with a heart of gold but an extraordinary knack for getting into hilarious and unpredictable situations.\n\nOne sunny morning, Goofy woke up with an idea buzzing in his head. He wanted to surprise his best friends, Mickey and Donald, with a special homemade picnic. With his characteristic enthusiasm, he gathered all the ingredients and set off to the beautiful countryside.\n\nAs Goofy strolled through the fields, he couldn't help but notice a colorful kite soaring high above him. Fascinated, he decided to give it a try. Without hesitation, he grabbed the kite and started running, hoping to feel the thrill of flying like the kite. However, his clumsiness got the best of him, and he ended up tripping over his own feet, sending the kite spiraling towards a nearby tree. Goofy's face turned beet red as he sheepishly retr

# Execute custom functions.

The RunnableLambda, within LangChain, serves as an abstraction enabling the transformation of custom Python functions into pipe functions.

This example bears similarity to the Runnable class introduced earlier in the article.







In [ ]:
from langchain_core.runnables import RunnableLambda

def add_one(x):
    return x + 1

def add_two(x):
    return x + 2

# wrap the functions with RunnableLambda
add_one = RunnableLambda(add_one)
add_two = RunnableLambda(add_two)
chain = add_one | add_two
chain.invoke(0)

3

**Conclusion**
As we conclude this examination, it's essential to recognize that the Langchain Expression Language (LCEL) spans beyond the subjects we've covered. LCEL effortlessly delves into diverse domains, encompassing Conversational Retrieval Chains, Multi-LLM Chain Fusion, Tools Integration, Memory Enhancement, SQL Querying, and Python REPL Coding, showcasing its versatility and broad functionality.
